In [ ]:
from utils.power_api import fetch_power_api
import pandas as pd
import numpy as np
from datetime import datetime

: 

In [ ]:
# =======================
# Main analysis function
# =======================
def analysis(data):
    raw = fetch(data["location"]["lat"], data["location"]["lon"])
    df = pandify(raw['properties']['parameter'])
    df_scoped = scope(df, data["date_range"]["start"], data["date_range"]["end"])

    comfort_risk = {}
    for activity in data["activities"]:
        probs = probabilities(df_scoped, activity, data["location"]["name"])
        colors = color_coding(probs)
        comfort_risk[activity] = {"probabilities": probs, "color_codes": colors}

    return {
        "date_range": data["date_range"],
        "location": data["location"],
        "activities": data["activities"],
        "comfort_risk": comfort_risk,
        "guidance": "Green: Good to go! Blue: Fair. Yellow: Caution. Red: Consider plan B.",
    }

In [ ]:
# =======================
# NASA POWER fetch
# =======================
def fetch(lat, lon):
    return fetch_power_api(
        "daily",
        2020,
        2025,
        lat,
        lon,
        ["T2M", "PRECTOT", "WS10M", "RH2M"],
    )

In [ ]:
# =======================
# Data cleanup helpers
# =======================
def pandify(data: dict) -> pd.DataFrame:
    df = pd.DataFrame(data)
    df = df.replace(-999.0, np.nan).dropna()
    df.index = pd.to_datetime(df.index, format="%Y%m%d")
    return df


def scope(df: pd.DataFrame, start: str, end: str) -> pd.DataFrame:
    start_dt = datetime.strptime(start, "%Y-%m-%d")
    end_dt = datetime.strptime(end, "%Y-%m-%d")
    start_md = (start_dt.month, start_dt.day)
    end_md = (end_dt.month, end_dt.day)

    mask = (
        (df.index.month > start_md[0])
        | ((df.index.month == start_md[0]) & (df.index.day >= start_md[1]))
    ) & (
        (df.index.month < end_md[0])
        | ((df.index.month == end_md[0]) & (df.index.day <= end_md[1]))
    )

    return df.loc[mask]

In [ ]:

# =======================
# Probabilities + Comfort Score
# =======================
def probabilities(df: pd.DataFrame, activity: str, location: str, start_date: str, end_date: str) -> dict:
    thresholds = {
        "hiking": {"temp": [10,25], "wind speed": [0,20], "humidity": [30,60], "precipitation": [0,2]},
        "paragliding": {"temp": [15,28], "wind speed": [10,25], "humidity": [30,65], "precipitation": [0,1]},
        "fishing": {"temp": [10,30], "wind speed": [0,25], "humidity": [30,80], "precipitation": [0,3]},
    }

    th = thresholds.get(activity, thresholds["hiking"])
    params = ["temp", "wind speed", "humidity", "precipitation"]

    # Convert dates
    start_dt = pd.to_datetime(start_date)
    end_dt = pd.to_datetime(end_date)

    # Case 1: Single-day activity → analyze by hours
    if start_dt.date() == end_dt.date():
        df_period = df.loc[start_dt:end_dt]  # Expect hourly data from NASA
        if df_period.empty:
            return {"error": "No hourly data available for this day"}

        return _analyze(df_period, th, params, unit="hour")

    # Case 2: Multi-day activity → analyze by days
    else:
        df_period = df.loc[start_dt:end_dt]  # Expect daily data from NASA
        if df_period.empty:
            return {"error": "No daily data available for this period"}

        return _analyze(df_period, th, params, unit="day")


# -----------------------
# Analysis Helper
# -----------------------
def _analyze(df_period: pd.DataFrame, th: dict, params: list, unit: str = "day") -> dict:
    """Perform quartile, mean, ranking analysis on either daily or hourly data."""
    # ----------------------------
    # Step 1: Quartile Analysis
    # ----------------------------
    quartiles = {}
    for param in params:
        series = df_period[param]
        q1, q3 = series.quantile(0.25), series.quantile(0.75)
        quartiles[param] = {"Q1": q1, "Q3": q3}

    # Filter inside IQR
    mask = pd.Series(True, index=df_period.index)
    for param in params:
        q1, q3 = quartiles[param]["Q1"], quartiles[param]["Q3"]
        mask &= (df_period[param] >= q1) & (df_period[param] <= q3)
    df_iqr = df_period.loc[mask]

    # Daily/hourly comfort scores
    scores = {}
    for timestamp, row in df_iqr.iterrows():
        raw_score = 0
        for param in params:
            min_val, max_val = th[param]
            val = row[param]
            raw_score += 1 if min_val <= val <= max_val else -1

        n = len(params)
        comfort_score_pct = ((raw_score - (-n)) / (2*n)) * 100
        scores[str(timestamp)] = {
            "values": row[params].to_dict(),
            "raw_score": raw_score,
            "comfort_score_pct": round(comfort_score_pct, 1),
            "score_bar": make_bar(comfort_score_pct),
            "color": comfort_color(comfort_score_pct),
        }

    # Rank all entries (days or hours)
    ranked = dict(
        sorted(scores.items(), key=lambda x: x[1]["comfort_score_pct"], reverse=True)
    )

    best = dict(list(ranked.items())[:2])
    worst = dict(list(ranked.items())[-2:])

    # ----------------------------
    # Step 2: Mean Analysis
    # ----------------------------
    means = df_period[params].mean().to_dict()
    raw_score_mean = 0
    for param in params:
        min_val, max_val = th[param]
        val = means[param]
        raw_score_mean += 1 if min_val <= val <= max_val else -1

    n = len(params)
    comfort_score_pct_mean = ((raw_score_mean - (-n)) / (2*n)) * 100
    color = comfort_color(comfort_score_pct_mean)
    suggestion = comfort_suggestion(color)

    mean_summary = {
        "means": means,
        "raw_score": raw_score_mean,
        "comfort_score_pct": round(comfort_score_pct_mean, 1),
        "score_bar": make_bar(comfort_score_pct_mean),
        "color": color,
        "suggestion": suggestion,
    }

    return {
        f"{unit}_analysis": {
            "quartiles": quartiles,
            "scores": scores,
            "ranked": ranked,
            "best": best,
            "worst": worst,
        },
        "mean_analysis": mean_summary,
    }


# -----------------------
# Helpers
# -----------------------
def comfort_color(pct: float) -> str:
    if pct >= 100:
        return "green"
    elif pct >= 75:
        return "yellow"
    elif pct >= 50:
        return "orange"
    else:
        return "red"


def comfort_suggestion(color: str) -> str:
    mapping = {
        "green": "Comfortable",
        "yellow": "Safe",
        "orange": "Moderate",
        "red": "Risky"
    }
    return mapping.get(color, "Unknown")


def make_bar(pct: float, length: int = 20) -> str:
    """Create a text-based bar for percentage values."""
    filled = int((pct / 100) * length)
    return "█" * filled + "░" * (length - filled) + f" {pct:.1f}%"


In [ ]:
# ------------------------
# Libraries
# ------------------------
import pandas as pd
import numpy as np
from datetime import datetime
from utils.power_api import fetch_power_api  # your existing NASA API fetch function

# ------------------------
# Activity Location Coordinates
# ------------------------
location_coords = {
    "Chittagong": {"lat": 22.3569, "lon": 91.7832},
    "Sylhet": {"lat": 24.8949, "lon": 91.8687},
    # Add more locations as needed
}

# ------------------------
# Main Probabilities Function
# ------------------------
def probabilities(activity: str, location: str, start_date: str, end_date: str) -> dict:
    thresholds = {
        "hiking": {"temp": [10,25], "wind speed": [0,20], "humidity": [30,60], "precipitation": [0,2]},
        "paragliding": {"temp": [15,28], "wind speed": [10,25], "humidity": [30,65], "precipitation": [0,1]},
        "fishing": {"temp": [10,30], "wind speed": [0,25], "humidity": [30,80], "precipitation": [0,3]},
    }

    th = thresholds.get(activity, thresholds["hiking"])
    params = ["temp", "wind speed", "humidity", "precipitation"]

    # Convert dates
    start_dt = pd.to_datetime(start_date)
    end_dt = pd.to_datetime(end_date)

    # ------------------------------
    # Step 1: Fetch NASA POWER data
    # ------------------------------
    df = fetch_power_api_data(location, start_date, end_date)
    if df.empty:
        return {"error": "No data returned from NASA for this location/date range"}

    # ------------------------------
    # Step 2: Determine hourly or daily
    # ------------------------------
    unit = "hour" if start_dt.date() == end_dt.date() else "day"
    df_period = df

    # ------------------------------
    # Step 3: Analyze
    # ------------------------------
    return _analyze(df_period, th, params, unit=unit)

# ------------------------
# Analysis Helper
# ------------------------
def _analyze(df_period: pd.DataFrame, th: dict, params: list, unit: str = "day") -> dict:
    # ----------------------------
    # Quartile Analysis
    # ----------------------------
    quartiles = {}
    for param in params:
        series = df_period[param]
        q1, q3 = series.quantile(0.25), series.quantile(0.75)
        quartiles[param] = {"Q1": q1, "Q3": q3}

    # Filter inside IQR
    mask = pd.Series(True, index=df_period.index)
    for param in params:
        q1, q3 = quartiles[param]["Q1"], quartiles[param]["Q3"]
        mask &= (df_period[param] >= q1) & (df_period[param] <= q3)
    df_iqr = df_period.loc[mask]

    # Compute comfort scores
    scores = {}
    for timestamp, row in df_iqr.iterrows():
        raw_score = 0
        for param in params:
            min_val, max_val = th[param]
            val = row[param]
            raw_score += 1 if min_val <= val <= max_val else -1

        n = len(params)
        comfort_score_pct = ((raw_score - (-n)) / (2*n)) * 100
        scores[str(timestamp)] = {
            "values": row[params].to_dict(),
            "raw_score": raw_score,
            "comfort_score_pct": round(comfort_score_pct, 1),
            "score_bar": make_bar(comfort_score_pct),
            "color": comfort_color(comfort_score_pct),
        }

    # Rank all entries
    ranked = dict(sorted(scores.items(), key=lambda x: x[1]["comfort_score_pct"], reverse=True))
    best = dict(list(ranked.items())[:2])
    worst = dict(list(ranked.items())[-2:])

    # ----------------------------
    # Mean Analysis
    # ----------------------------
    means = df_period[params].mean().to_dict()
    raw_score_mean = 0
    for param in params:
        min_val, max_val = th[param]
        val = means[param]
        raw_score_mean += 1 if min_val <= val <= max_val else -1

    n = len(params)
    comfort_score_pct_mean = ((raw_score_mean - (-n)) / (2*n)) * 100
    color = comfort_color(comfort_score_pct_mean)
    suggestion = comfort_suggestion(color)

    mean_summary = {
        "means": means,
        "raw_score": raw_score_mean,
        "comfort_score_pct": round(comfort_score_pct_mean, 1),
        "score_bar": make_bar(comfort_score_pct_mean),
        "color": color,
        "suggestion": suggestion,
    }

    return {
        f"{unit}_analysis": {
            "quartiles": quartiles,
            "scores": scores,
            "ranked": ranked,
            "best": best,
            "worst": worst,
        },
        "mean_analysis": mean_summary,
    }

# ------------------------
# Helper: NASA POWER API Fetch
# ------------------------
def fetch_power_api_data(location: str, start_date: str, end_date: str) -> pd.DataFrame:
    coords = location_coords.get(location)
    if not coords:
        return pd.DataFrame()  # return empty if location not found

    # Choose daily or hourly
    time_interval = "hourly" if start_date == end_date else "daily"

    raw = fetch_power_api(
        time_interval,
        int(start_date[:4]),
        int(end_date[:4]),
        coords["lat"],
        coords["lon"],
        ["T2M", "PRECTOT", "WS10M", "RH2M"]
    )

    df = pd.DataFrame(raw["properties"]["parameter"]).T
    df.index = pd.to_datetime(df.index, format="%Y%m%d")
    df.replace(-999.0, np.nan, inplace=True)
    df.dropna(inplace=True)
    df.rename(columns={"T2M": "temp", "PRECTOT": "precipitation", "WS10M": "wind speed", "RH2M": "humidity"}, inplace=True)
    return df

# ------------------------
# Helper: Comfort Color
# ------------------------
def comfort_color(pct: float) -> str:
    if pct >= 100:
        return "green"
    elif pct >= 75:
        return "yellow"
    elif pct >= 50:
        return "orange"
    else:
        return "red"

# ------------------------
# Helper: Suggestion
# ------------------------
def comfort_suggestion(color: str) -> str:
    mapping = {
        "green": "Comfortable",
        "yellow": "Safe",
        "orange": "Moderate",
        "red": "Risky"
    }
    return mapping.get(color, "Unknown")

# ------------------------
# Helper: Text-based Score Bar
# ------------------------
def make_bar(pct: float, length: int = 20) -> str:
    filled = int((pct / 100) * length)
    return "█" * filled + "░" * (length - filled) + f" {pct:.1f}%"
